In [1]:
import plotly
import plotly.express as px
plotly.__version__

'5.18.0'

In [20]:
import matplotlib.pyplot as plt


In [3]:
import pandas as pd
churn_data = pd.read_csv('data/churn.csv')
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


* 9.1. Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

* 9.2. Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

* 9.3. Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

* 9.4. Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

* 9.5. Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

* 9.6. Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

9.1 Соотношение клиентов

In [44]:
counts = churn_data['Exited'].value_counts()
count = pd.DataFrame(data = [[0 , 7963 ],[1 , 2037]],
                     columns= ['Exited', 'count'])
count

fig = px.bar(
    data_frame=count,
    x='Exited',
    y='count'
)
fig.show()

#fig.show()
#counts.plot(kind='bar')
#plt.xlabel('Status')
#plt.ylabel('Count')
#plt.title('Churn Data')
#plt.show()
#print (counts)

Колличество лояльных клиентов банка в 3,9 раза превышает колличество ушедших клиентов.

    9.2. Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [22]:
counts = churn_data['Balance'].value_counts()
counts


Balance
0.00         3617
130170.82       2
105473.74       2
85304.27        1
159397.75       1
             ... 
81556.89        1
112687.69       1
108698.96       1
238387.56       1
130142.79       1
Name: count, Length: 6382, dtype: int64

In [ ]:
fig = px.histogram(
    balance,
    x='Balance',
    labels={'Balance':'Баланс на счету клиента, $', 'y':'Количество клиентов, чел.'},
    marginal='box',
    nbins=30,
